In [1]:
import numpy as np
import pandas as pd
import pickle
import requests

In [2]:
df = pd.read_csv("AER_credit_card_data.csv")
df.head(2)

,card,reports,age,income,share,expenditure,owner,selfemp,dependents,months,majorcards,active
0,yes,0,37.66667,4.52,0.033270,124.983300,yes,no,3,54,1,12
1,yes,0,33.25000,2.42,0.005217,9.854167,no,no,3,34,1,13


In [3]:
# !pip install pipenv
!pipenv --version

pipenv, version 2022.9.24


####  Q1. What's the version of pipenv you installed?

#### 2022.9.24

Use Pipenv to install Scikit-Learn version 1.0.2

In [4]:
# !pipenv install scikit-learn==1.0.2

!cat Pipfile.lock | jq '.default."scikit-learn".hashes[0]'

"sha256:08ef968f6b72033c16c479c966bf37ccd49b06ea91b765e1cc27afefe723920b"


#### Q2. What's the first hash for scikit-learn you get in Pipfile.lock?

#### sha256:08ef968f6b72033c16c479c966bf37ccd49b06ea91b765e1cc27afefe723920b

Models

We've prepared a dictionary vectorizer and a model.

They were trained (roughly) using this code:

features = ['reports', 'share', 'expenditure', 'owner']
dicts = df[features].to_dict(orient='records')

dv = DictVectorizer(sparse=False)
X = dv.fit_transform(dicts)

model = LogisticRegression(solver='liblinear').fit(X, y)

    Note: You don't need to train the model. This code is just for your reference.

And then saved with Pickle. Download them:

    DictVectorizer
    LogisticRegression

With wget

In [5]:
# PREFIX="https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/course-zoomcamp/cohorts/2022/05-deployment/homework"
# !wget $PREFIX/model1.bin
# !wget $PREFIX/dv.bin

Let's use these models!

    Write a script for loading these models with pickle
    Score this client:

{"reports": 0, "share": 0.001694, "expenditure": 0.12, "owner": "yes"}

In [6]:
with open('model1.bin', 'rb') as f_in:
    model = pickle.load(f_in)
    
with open('dv.bin', 'rb') as f_in:
    dv = pickle.load(f_in)

In [7]:
client1 = {"reports": 0, "share": 0.001694, "expenditure": 0.12, "owner": "yes"}

print(round(model.predict_proba(dv.transform([client1]))[0][1], 3))

0.162


#### Q3. What's the probability that this client will get a credit card?

#### 0.162

Now let's serve this model as a web service

    Install Flask and gunicorn
    Write Flask code for serving the model
    Now score this client using requests:

url = "YOUR_URL"
client = {"reports": 0, "share": 0.245, "expenditure": 3.438, "owner": "yes"}
requests.post(url, json=client).json()

In [8]:
url = 'http://localhost:5000/predict'
client = {"reports": 0, "share": 0.245, "expenditure": 3.438, "owner": "yes"}

response = requests.post(url, json=client).json()
print(response['credit_acceptance_probability'])

0.928


#### Q4. What's the probability that this client will get a credit card?

#### 0.928

Download the base image svizor/zoomcamp-model:3.9.12-slim. You can easily make it by using docker pull command.

In [9]:
# compressed size of layers (including multi-arch image(s), if any)
# !docker manifest inspect svizor/zoomcamp-model:3.9.12-slim | grep 'size' | awk -F ':' '{sum+=$NF} ENDx {print sum}' | numfmt --to=iec

# !docker inspect svizor/zoomcamp-model:3.9.12-slim | jq '.[].Size' | numfmt --to=iec

In [10]:
# !docker pull svizor/zoomcamp-model:3.9.12-slim
!docker images svizor/zoomcamp-model:3.9.12-slim | awk '{ print $7 }'


125MB


#### Q5. So what's the size of this base image?

#### 125 MB

Now create your own Dockerfile based on the image we prepared.

It should start like that:

FROM svizor/zoomcamp-model:3.9.12-slim

Now complete it:

    Install all the dependencies from the Pipenv file
    Copy your Flask script
    Run it with Gunicorn

After that, you can build your docker image.

Let's run your docker container!

After running it, score this client once again:

In [11]:
url = 'http://localhost:5000/predict'

client = {"reports": 0, "share": 0.245, "expenditure": 3.438, "owner": "yes"}
response = requests.post(url, json=client).json()

print(response['credit_acceptance_probability'])

0.928


#### Q6. What's the probability that this client will get a credit card now?

#### 0.928